In [58]:
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import cross_validation, metrics
from sklearn.grid_search import GridSearchCV

import matplotlib.pylab as plt
%matplotlib inline

In [59]:
trainf='/Users/wyatt/Documents/Docs-wMBP15/SAS/stk_model/model.Wyatt_stock_train.csv'
#valid='/Users/wyatt/Documents/Docs-wMBP15/SAS/stk_model/model.Wyatt_stock_valid.csv'
train=pd.read_csv(trainf).fillna(0)
target='response' # Disbursed的值就是二元分类的输出
IDcol = 'cust_id'
train['response'].value_counts() 

0    13872
1      694
Name: response, dtype: int64

In [44]:
x_columns = [x for x in train.columns if x not in [target, IDcol]]
X = train[x_columns]
y = train[target]

In [45]:
gbm0 = GradientBoostingClassifier(random_state=10)
gbm0.fit(X,y)
y_pred = gbm0.predict(X)
y_predprob = gbm0.predict_proba(X)[:,1]
print("Accuracy : %.4g" % metrics.accuracy_score(y.values, y_pred))
print("AUC Score (Train): %f" % metrics.roc_auc_score(y, y_predprob))

Accuracy : 0.961
AUC Score (Train): 0.911319


In [46]:
param_test1 = {'n_estimators':list(range(80,121,10))}
gsearch1 = GridSearchCV(estimator = GradientBoostingClassifier(learning_rate=0.01, min_samples_split=300,
                                  min_samples_leaf=20,max_depth=8,max_features='sqrt', subsample=0.8,random_state=10), 
                       param_grid = param_test1, scoring='roc_auc',iid=False,cv=5)
gsearch1.fit(X,y)
gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_

([mean: 0.78366, std: 0.01986, params: {'n_estimators': 80},
  mean: 0.78339, std: 0.01950, params: {'n_estimators': 90},
  mean: 0.78440, std: 0.01985, params: {'n_estimators': 100},
  mean: 0.78486, std: 0.01981, params: {'n_estimators': 110},
  mean: 0.78523, std: 0.01970, params: {'n_estimators': 120}],
 {'n_estimators': 120},
 0.7852250362250219)

In [55]:
n_estimators = gsearch1.best_params_['n_estimators']

In [56]:
param_test2 = {'max_depth':list(range(3,9,2)), 'min_samples_split':list(range(50,201,20))}
gsearch2 = GridSearchCV(estimator = GradientBoostingClassifier(learning_rate=0.1, n_estimators=n_estimators, min_samples_leaf=20, 
      max_features='sqrt', subsample=0.8, random_state=10), 
   param_grid = param_test2, scoring='roc_auc',iid=False, cv=5)
gsearch2.fit(X,y)
gsearch2.grid_scores_, gsearch2.best_params_, gsearch2.best_score_

([mean: 0.78795, std: 0.01818, params: {'max_depth': 3, 'min_samples_split': 50},
  mean: 0.78542, std: 0.02113, params: {'max_depth': 3, 'min_samples_split': 70},
  mean: 0.78710, std: 0.02115, params: {'max_depth': 3, 'min_samples_split': 90},
  mean: 0.78629, std: 0.02010, params: {'max_depth': 3, 'min_samples_split': 110},
  mean: 0.78539, std: 0.02049, params: {'max_depth': 3, 'min_samples_split': 130},
  mean: 0.78478, std: 0.02141, params: {'max_depth': 3, 'min_samples_split': 150},
  mean: 0.78501, std: 0.02189, params: {'max_depth': 3, 'min_samples_split': 170},
  mean: 0.78851, std: 0.02030, params: {'max_depth': 3, 'min_samples_split': 190},
  mean: 0.78514, std: 0.02421, params: {'max_depth': 5, 'min_samples_split': 50},
  mean: 0.78505, std: 0.02029, params: {'max_depth': 5, 'min_samples_split': 70},
  mean: 0.78571, std: 0.02320, params: {'max_depth': 5, 'min_samples_split': 90},
  mean: 0.78307, std: 0.02462, params: {'max_depth': 5, 'min_samples_split': 110},
  mean: 0.

In [23]:
param_test3 = {'min_samples_split':list(range(1400,1600,100)), 'min_samples_leaf':list(range(140,171,10))}
gsearch3 = GridSearchCV(estimator = GradientBoostingClassifier(learning_rate=0.1, n_estimators=120,max_depth=9,
                                     max_features='sqrt', subsample=0.8, random_state=10), 
                       param_grid = param_test3, scoring='roc_auc',iid=False, cv=5)
gsearch3.fit(X,y)
gsearch3.grid_scores_, gsearch3.best_params_, gsearch3.best_score_

([mean: 0.77406, std: 0.04680, params: {'min_samples_leaf': 140, 'min_samples_split': 1400},
  mean: 0.76972, std: 0.04591, params: {'min_samples_leaf': 140, 'min_samples_split': 1500},
  mean: 0.76986, std: 0.05525, params: {'min_samples_leaf': 150, 'min_samples_split': 1400},
  mean: 0.78831, std: 0.04916, params: {'min_samples_leaf': 150, 'min_samples_split': 1500},
  mean: 0.78224, std: 0.05331, params: {'min_samples_leaf': 160, 'min_samples_split': 1400},
  mean: 0.77226, std: 0.04056, params: {'min_samples_leaf': 160, 'min_samples_split': 1500},
  mean: 0.77860, std: 0.04658, params: {'min_samples_leaf': 170, 'min_samples_split': 1400},
  mean: 0.78099, std: 0.05406, params: {'min_samples_leaf': 170, 'min_samples_split': 1500}],
 {'min_samples_leaf': 150, 'min_samples_split': 1500},
 0.7883117966393429)

In [24]:
gbm1 = GradientBoostingClassifier(learning_rate=0.1, n_estimators=120,max_depth=9, min_samples_leaf =150, 
               min_samples_split =1500, max_features='sqrt', subsample=0.8, random_state=10)
gbm1.fit(X,y)
y_pred = gbm1.predict(X)
y_predprob = gbm1.predict_proba(X)[:,1]
print( "Accuracy : %.4g" % metrics.accuracy_score(y.values, y_pred))
print ("AUC Score (Train): %f" % metrics.roc_auc_score(y, y_predprob))

Accuracy : 0.9933
AUC Score (Train): 0.998991


In [25]:
param_test4 = {'max_features':list(range(7,20,2))}
gsearch4 = GridSearchCV(estimator = GradientBoostingClassifier(learning_rate=0.1, n_estimators=120,max_depth=9, min_samples_leaf =150, 
               min_samples_split =1500, subsample=0.8, random_state=10), 
                       param_grid = param_test4, scoring='roc_auc',iid=False, cv=5)
gsearch4.fit(X,y)
gsearch4.grid_scores_, gsearch4.best_params_, gsearch4.best_score_

([mean: 0.75640, std: 0.05208, params: {'max_features': 7},
  mean: 0.76820, std: 0.05650, params: {'max_features': 9},
  mean: 0.77248, std: 0.05077, params: {'max_features': 11},
  mean: 0.78831, std: 0.04916, params: {'max_features': 13},
  mean: 0.77366, std: 0.05624, params: {'max_features': 15},
  mean: 0.77685, std: 0.06260, params: {'max_features': 17},
  mean: 0.76468, std: 0.05720, params: {'max_features': 19}],
 {'max_features': 13},
 0.7883117966393429)

In [ ]:
param_test5 = {'subsample':[0.6,0.7,0.75,0.8,0.85,0.9]}
gsearch5 = GridSearchCV(estimator = GradientBoostingClassifier(learning_rate=0.1, n_estimators=60,max_depth=3, min_samples_leaf =60, 
               min_samples_split =1400, max_features=11, random_state=10), 
                       param_grid = param_test5, scoring='roc_auc',iid=False, cv=5)
gsearch5.fit(X,y)
gsearch5.grid_scores_, gsearch5.best_params_, gsearch5.best_score_

In [27]:
gbm2 = GradientBoostingClassifier(learning_rate=0.05, n_estimators=240,max_depth=9, min_samples_leaf =150, 
               min_samples_split =1500, max_features='sqrt', subsample=0.8, random_state=10)
gbm2.fit(X,y)
y_pred = gbm2.predict(X)
y_predprob = gbm2.predict_proba(X)[:,1]
print( "Accuracy : %.4g" % metrics.accuracy_score(y.values, y_pred))
print ("AUC Score (Train): %f" % metrics.roc_auc_score(y, y_predprob))

Accuracy : 0.9931
AUC Score (Train): 0.999171


In [29]:
gbm2 = GradientBoostingClassifier(learning_rate=0.1, n_estimators=120,max_depth=9, min_samples_leaf =150, 
               min_samples_split =1500, max_features='sqrt', subsample=0.8, random_state=10)
gbm2.fit(X,y)
y_pred = gbm2.predict(X)
y_predprob = gbm2.predict_proba(X)[:,1]
print( "Accuracy : %.4g" % metrics.accuracy_score(y.values, y_pred))
print ("AUC Score (Train): %f" % metrics.roc_auc_score(y, y_predprob))

Accuracy : 0.9933
AUC Score (Train): 0.998991


In [30]:
import sklearn

In [ ]:
sklearn.ensemble.GradientBoostingClassifier.